# Job Postings Data Cleaning

## Import Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/team-25/

!pip install langdetect -q

import pandas as pd
import numpy as np
from langdetect import detect
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
[csv_file for csv_file in os.listdir("./data") if csv_file.endswith(".csv")]

## Import Raw Data

In [28]:
filelist = ['DataAnalyst.csv', 'DataScientist.csv', 'DataEngineer.csv']
df_list=[]
for filename in filelist:
  df = pd.read_csv(f"./data/{filename}")
  df["job_family"] = filename.replace(".csv","")
  df_list.append(df)

df = pd.concat(df_list)
df.drop(columns=df.columns[0], inplace=True)
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,job_family,index
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,DataAnalyst,NaN
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1,DataAnalyst,NaN
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1,DataAnalyst,NaN
3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1,DataAnalyst,NaN
4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True,DataAnalyst,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,$76K-$128K (Glassdoor est.),The Expertise You Have Proven expertise in dev...,4.5,PMG Global\n4.5,"Westlake, TX","Herndon, VA",1 to 50 employees,-1,Company - Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD),-1,-1,DataEngineer,NaN
2524,Electrical Engineer,$76K-$128K (Glassdoor est.),Lockheed Martin is seeking a Circuit and FPGA ...,2.9,Albin Engineering Services\n2.9,"Fort Worth, TX","Santa Clara, CA",51 to 200 employees,1993,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1,-1,DataEngineer,NaN
2525,Sr. Big Data Engineer,$76K-$128K (Glassdoor est.),Posting Title Principal Data Engineer (Contrac...,3.7,Cincinnati Bell Technology Solutions\n3.7,"Irving, TX","Cincinnati, OH",501 to 1000 employees,-1,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),-1,-1,DataEngineer,NaN
2526,Senior Big Data / ETL Engineer,$76K-$128K (Glassdoor est.),"Your Opportunity\n\nCharles Schwab & Co., Inc ...",3.5,Schwab\n3.5,"Westlake, TX","Dettenheim, Germany",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable,-1,-1,DataEngineer,NaN


## Basic Data Cleaning

In [29]:
df.drop(columns=["Competitors", "Easy Apply", "Headquarters","index"], inplace= True)

# converts salary estimate from looking like "$10K - $20K (Glassdoor Est....)" to 10,000 in lower bound column and 20,000 in upper bound column
df["Salary_Estimate_Lower_Bound"] = df["Salary Estimate"].apply(lambda x: x.split("-")[0] if x != "-1" else "$0")
df["Salary_Estimate_Upper_Bound"] = df["Salary Estimate"].apply(lambda x: x.split("-")[1].split(" (")[0] if x != "-1" else "$0" )
# converts salary estimate from looking like "$10K - $20K (Glassdoor Est....)" to 10,000 in lower bound column and 20,000 in upper bound column
df["Salary_Estimate_Lower_Bound"] = df["Salary Estimate"].apply(lambda x: x.split("-")[0] if x != "-1" else "$0")+"H"
df["Salary_Estimate_Upper_Bound"] = df["Salary Estimate"].apply(lambda x: x.split("-")[1].split(" ")[0].split("(")[0] if x != "-1" else "$0" )+"H"
df["Salary_Estimate_Lower_Bound"] = df["Salary_Estimate_Lower_Bound"].str.strip("$").replace({"KH":"*1e3", "MH":"*1e6", "H": "*2080"}, regex=True).map(pd.eval).astype(int)
df["Salary_Estimate_Upper_Bound"] = df["Salary_Estimate_Upper_Bound"].str.strip("$").replace({"KH":"*1e3", "MH":"*1e6", "H": "*2080"}, regex=True).map(pd.eval).astype(int)
df.drop(index = df.loc[ df["Salary Estimate"] =="-1"].index, inplace=True) # only 1 row
df.drop(columns = ["Salary Estimate"] , inplace=True) # already have this

# cleaning company data
df.drop(index = df [ ((df["Company Name"]=="-1") | (df["Company Name"].isna()))].index, inplace=True) # only 1 row
df["Company Name"] = df["Company Name"].apply(lambda x : x.split("\n")[0]) # necessary because rating seems to get attached to end of company name

# # get rid of non-english postings
df["language_code"] = df["Job Description"].apply(lambda x: detect(x[:200]))
df.drop(index = df[df["language_code"]!= "en"].index, inplace=True) # only 3 entries

# addition cleaning putting -1 to unknown or n/a
df.loc[ df.Size == "-1", "Size"] = "Unknown"
df.loc[ df["Type of ownership"] == "-1", "Type of ownership"] = "Unknown"
df.loc[ df["Revenue"] == "-1", "Revenue"] = "Unknown / Non-Applicable"
df.loc[ df["Industry"] == "-1", "Industry"] = "Unknown"
df.loc[ df["Sector"] == "-1", "Sector"] = "Unknown"

In [30]:
df

,Job Title,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_family,Salary_Estimate_Lower_Bound,Salary_Estimate_Upper_Bound,language_code
0,"Data Analyst, Center on Immigration and Justic...",Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),DataAnalyst,37000,66000,en
1,Quality Data Analyst,Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),DataAnalyst,37000,66000,en
2,"Senior Data Analyst, Insights & Analytics Team...",We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,DataAnalyst,37000,66000,en
3,Data Analyst,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),DataAnalyst,37000,66000,en
4,Reporting Data Analyst,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel,"New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DataAnalyst,37000,66000,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,The Expertise You Have Proven expertise in dev...,4.5,PMG Global,"Westlake, TX",1 to 50 employees,-1,Company - Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD),DataEngineer,76000,128000,en
2524,Electrical Engineer,Lockheed Martin is seeking a Circuit and FPGA ...,2.9,Albin Engineering Services,"Fort Worth, TX",51 to 200 employees,1993,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),DataEngineer,76000,128000,en
2525,Sr. Big Data Engineer,Posting Title Principal Data Engineer (Contrac...,3.7,Cincinnati Bell Technology Solutions,"Irving, TX",501 to 1000 employees,-1,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),DataEngineer,76000,128000,en
2526,Senior Big Data / ETL Engineer,"Your Opportunity\n\nCharles Schwab & Co., Inc ...",3.5,Schwab,"Westlake, TX",Unknown,-1,Company - Public,Unknown,Unknown,Unknown / Non-Applicable,DataEngineer,76000,128000,en


## Text Data Cleaning

In [32]:
def clean_text(text_string,lemmatize=False):
  # TODO: Rewrite function to decrease run-time.
  text_data = re.sub('[^a-zA-Z]', ' ', text_string)
  text_data = text_data.lower()
  text_data = text_data.split()
  clean_text = ' '.join(text_data)
  if lemmatize:
    wl = WordNetLemmatizer()
    text_data = [wl.lemmatize(word) for word in text_data if not word in set(stopwords.words('english'))]
    text_data = ' '.join(text_data)
    return text_data
  else:
    return clean_text

df["clean_job_description"] = df["Job Description"].apply(lambda x: clean_text(x))
df["lemmatized_job_description"] = df["Job Description"].apply(lambda x: clean_text(x, lemmatize=True))
df.drop(columns = ["language_code"], inplace=True)
df["clean_job_title"] = df["Job Title"].apply(lambda x: clean_text(x))
job_family_map = {"DataEngineer": "data engineer", "DataAnalyst": "data analyst", "DataScientist": "data scientist"}
df["job_family"] = df["job_family"].apply(lambda x: job_family_map[x])

In [36]:
df

,Job Title,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,job_family,Salary_Estimate_Lower_Bound,Salary_Estimate_Upper_Bound,clean_job_description,lemmatized_job_description,clean_job_title
0,"Data Analyst, Center on Immigration and Justic...",Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),data analyst,37000,66000,are you eager to roll up your sleeves and harn...,eager roll sleeve harness data drive policy ch...,data analyst center on immigration and justice...
1,Quality Data Analyst,Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),data analyst,37000,66000,overview provides analytical and technical sup...,overview provides analytical technical support...,quality data analyst
2,"Senior Data Analyst, Insights & Analytics Team...",We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,data analyst,37000,66000,we re looking for a senior data analyst who ha...,looking senior data analyst love mentorship da...,senior data analyst insights analytics team cu...
3,Data Analyst,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),data analyst,37000,66000,requisition numberrr remote yes we collaborate...,requisition numberrr remote yes collaborate cr...,data analyst
4,Reporting Data Analyst,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel,"New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),data analyst,37000,66000,about fanduel group fanduel group is a world c...,fanduel group fanduel group world class team b...,reporting data analyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,Cloud Engineer,The Expertise You Have Proven expertise in dev...,4.5,PMG Global,"Westlake, TX",1 to 50 employees,-1,Company - Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD),data engineer,76000,128000,the expertise you have proven expertise in dev...,expertise proven expertise developing applicat...,cloud engineer
2524,Electrical Engineer,Lockheed Martin is seeking a Circuit and FPGA ...,2.9,Albin Engineering Services,"Fort Worth, TX",51 to 200 employees,1993,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),data engineer,76000,128000,lockheed martin is seeking a circuit and fpga ...,lockheed martin seeking circuit fpga design el...,electrical engineer
2525,Sr. Big Data Engineer,Posting Title Principal Data Engineer (Contrac...,3.7,Cincinnati Bell Technology Solutions,"Irving, TX",501 to 1000 employees,-1,Company - Private,IT Services,Information Technology,$100 to $500 million (USD),data engineer,76000,128000,posting title principal data engineer contract...,posting title principal data engineer contract...,sr big data engineer
2526,Senior Big Data / ETL Engineer,"Your Opportunity\n\nCharles Schwab & Co., Inc ...",3.5,Schwab,"Westlake, TX",Unknown,-1,Company - Public,Unknown,Unknown,Unknown / Non-Applicable,data engineer,76000,128000,your opportunity charles schwab co inc is curr...,opportunity charles schwab co inc currently se...,senior big data etl engineer


In [37]:
df.loc[df.job_family == "data analyst"].to_csv("./data/data_analyst_job_postings_local.csv", index=False, sep="|")
df.loc[df.job_family == "data engineer"].to_csv("./data/data_engineer_job_postings_local.csv", index=False, sep="|")
df.loc[df.job_family == "data scientist"].to_csv("./data/data_scientist_job_postings_local.csv", index=False, sep="|")

In [40]:
df_out = df.reset_index()
df_out.drop(columns = df_out.columns[0],inplace=True)
df_out.index.names = ['job_id']
df_out.to_csv("./data/data_job_postings_clean.csv", sep='|', index=False)
